In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import math
folderpath = '/home/jian/BiglotsCode/outputs/'
lastweeksdate = '2017-11-25'
# Read till 2017-12-02 as cell 3

In [2]:
dfsales = pd.read_csv(folderpath + 'combinedsales'+ '2017-12-02' + '.csv',sep = '|',dtype = 'str')

In [3]:
dfsales['week_end_dt']=pd.to_datetime(dfsales['week_end_dt'])

In [4]:
dfsales=dfsales.sort_values(by=['week_end_dt','location_id'], ascending=[True,True])

In [6]:
dfsales['week_end_dt']=dfsales['week_end_dt'].apply(lambda x: str(x.date()))

In [7]:
dfsales['class_gross_sales_amt'].replace('?','0',inplace=True)
dfsales['class_gross_sales_amt']=dfsales['class_gross_sales_amt'].astype(float)

In [48]:
dfstore_1 = pd.read_table('/home/jian/BigLots/static_files/MediaStorm Data Extract - Store list.txt',
                        sep = '|',dtype = 'str')
dfstore_2 = pd.read_table('/home/jian/BigLots/static_files/MediaStormStoresList_0913.txt',
                        sep = '|',dtype = 'str')
dfstore_3 = pd.read_table('/home/jian/BigLots/static_files/MediaStormStoreList_Nov15.txt',
                        sep = '|',dtype = 'str')
dfstore_1=dfstore_1[['location_id','zip_cd']]
dfstore_2=dfstore_2[['location_id','zip_cd']]
dfstore_3=dfstore_3[['location_id','zip_cd']]
dfstore=pd.concat([dfstore_1,dfstore_2,dfstore_3])
dfstore['zip']=dfstore['zip_cd'].apply(lambda x: x[0:5])
del dfstore['zip_cd']
dfstore=dfstore.drop_duplicates()

In [49]:
dma = pd.read_csv('/home/jian/BiglotsCode/OtherInput/zipdmamapping.csv',dtype = 'str')
dma=dma.rename(columns={'zip_cd': 'zip'})

In [46]:
recliner_sales=dfsales[dfsales['class_code_id']=='60803']
recliner_sales=recliner_sales[['location_id','week_end_dt','class_gross_sales_amt']]

In [47]:
recliner_sales=pd.pivot_table(recliner_sales,values = 'class_gross_sales_amt', index='location_id', columns = 'week_end_dt').reset_index()

In [50]:
recliner_sales=pd.merge(recliner_sales,dfstore,on='location_id',how='left')
recliner_sales=pd.merge(recliner_sales,dma,on='zip',how='left')

In [51]:
recliner_sales.head()

,location_id,2015-05-23,2015-05-30,2015-06-06,2015-06-13,2015-06-20,2015-06-27,2015-07-04,2015-07-11,2015-07-18,...,2017-10-14,2017-10-21,2017-10-28,2017-11-04,2017-11-11,2017-11-18,2017-11-25,2017-12-02,zip,DMA
0,1,914.98,714.98,712.99,578.00,3372.04,594.18,579.97,1064.97,1445.97,...,1166.49,799.98,599.98,2739.95,1624.46,1591.23,6105.00,1204.99,43232,"COLUMBUS, OH"
1,1001,1429.97,2983.92,1648.97,3914.93,2271.96,158.00,3554.86,876.00,2832.89,...,507.99,2634.93,2866.43,2504.96,1222.06,344.98,6843.91,1329.97,37110,CHATTANOOGA
2,1003,NaN,179.00,NaN,249.00,1225.99,NaN,1739.98,NaN,513.00,...,225.00,NaN,285.00,285.00,1425.00,NaN,4155.00,541.50,30120,ATLANTA
3,1005,1184.97,2929.91,584.99,1959.94,2068.29,684.99,1904.92,1597.94,1943.15,...,1444.98,1537.98,862.99,1873.55,927.98,1150.98,5112.16,955.00,27295,GREENSBORO-H.POINT-W.SALEM
4,1006,NaN,2412.95,819.97,2084.96,475.00,684.99,1959.91,1295.97,2720.94,...,1134.99,2253.12,594.99,1941.46,1134.97,1827.94,2303.97,209.30,29577,MYRTLE BEACH-FLORENCE


In [52]:
fireplace_sales=dfsales[dfsales['class_code_id']=='61204']
fireplace_sales=fireplace_sales[['location_id','week_end_dt','class_gross_sales_amt']]

In [53]:
fireplace_sales=pd.pivot_table(fireplace_sales,values = 'class_gross_sales_amt', index='location_id', columns = 'week_end_dt').reset_index()

In [54]:
fireplace_sales=pd.merge(fireplace_sales,dfstore,on='location_id',how='left')
fireplace_sales=pd.merge(fireplace_sales,dma,on='zip',how='left')

In [56]:
writer = pd.ExcelWriter('/home/jian/Projects/Big Lots/fireplace and reliner.xlsx',engine='xlsxwriter')

In [57]:
fireplace_sales.to_excel(writer,'fireplace')
recliner_sales.to_excel(writer,'recliner')

In [58]:
writer.save()